In [2]:
import girder_client
import os
import numpy as np
import matplotlib.pyplot as plt
import large_image
from shapely.geometry.polygon import Polygon
from shapely import GeometryCollection
#https://www.uv.es/gonmagar/blog/2018/11/11/RasterioExample
%matplotlib inline
## https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
#https://digitalslidearchive.github.io/HistomicsTK/examples/using_large_image.html
## Girder Client / large Image API info
gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
_ = gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [3]:
#refImage is online at styx.neurology.emory.edu .. I downloaded it already for testing..
sampleItem = '63cefc614da1ec8c4f65fe53'
ts = large_image.getTileSource("./Case 2.svs")
## Load the tile source
ts.getMetadata() 

/home/dagutman/.local/lib/python3.8/site-packages/glymur/config.py:153: UserWarning: The tiff library at /home/dagutman/.local/lib/python3.8/site-packages/Glymur.libs/libtiff-a107d8f8.so.6.0.0 could not be loaded.
  warnings.warn(msg, UserWarning)


{'levels': 9,
 'sizeX': 47808,
 'sizeY': 41741,
 'tileWidth': 240,
 'tileHeight': 240,
 'magnification': 20.0,
 'mm_x': 0.0005044,
 'mm_y': 0.0005044}

In [4]:
## First get the annotations available for the item of interst
availAnnotations = gc.get("annotation?itemId=%s" % sampleItem)
for a in availAnnotations:
    print(a['annotation'],a['_id'])

{'description': 'Shapes included regions we WANT to keep', 'name': 'dagRegionMask'} 63d02c944da1ec8c4f65fe5d


In [5]:
from pprint import pprint

## From above I only have one annotation, the dagRegionMask, I am just grabbing the itemId I want to grab the annotationObject
ao = gc.get("annotation/%s" % availAnnotations[0]['_id']) ## I am getting the first anntation object in this case
# pprint(ao)
## The key here to notice is the ['annotation']['elements'][0] which has the shape I am interested in.. then it's a set of points

roiPolygon = ao['annotation']['elements'][0]
roiPolygonPts = [(x,y) for (x,y,z) in roiPolygon['points']]
roiMask = Polygon(roiPolygonPts)

In [34]:
from shapely.affinity import affine_transform
import rasterio
import rasterio.features
from rasterio import Affine
from PIL import Image

md = ts.getMetadata()
baseSlidePoly = Polygon([(0,0),(0,md['sizeY']),(md['sizeX'],md['sizeY']),(md['sizeX'],0)])

tile_means = []
tile_areas = []
shapeSet = [roiMask]
tileIntersect = []
intersectionList = []
sink = large_image.new()

## Iterate in base pixels with a strike of 4000x4000 and return np arrays
tileSize=4000
num_tiles = 0

## Using 196 as background instead of 255
backgroundtile = np.ones([tileSize,tileSize],dtype=np.uint8)*243# Create RGBA Image that is all white

# np.ma.masked_where(roi_mask_p, tile_info['tile'])

for tile_info in ts.tileIterator(
    region=dict(units='base_pixels'),
    scale=dict(magnification=20),
    tile_size=dict(width=tileSize, height=tileSize),
    format=large_image.constants.TILE_FORMAT_NUMPY
):

    tix = tile_info['x'] ### Just an alias for tile_x and tile_y  ## which do not exist
    tiy = tile_info['y'] ### Just an alias for tile_x and tile_y
    tiw = tile_info['width']
    tih = tile_info['height']
    
    tileBounds = [(tix,tiy), (tix,tiy+tih), ((tix+tiw),(tiy+tih)),(tix+tiw,tiy)]
    tilePoly = Polygon(tileBounds)

    if roiMask.contains(tilePoly):
        ## Do not need a mask, entire ROI is within the shape
        sink.addTile( tile_info['tile'],tix,tiy  )

        #print("The objects are bounded..",tileBounds)
    elif roiMask.intersects(tilePoly):
        tileIntersect.append( tilePoly)
        intersectionList.append( (roiMask.intersection(tilePoly),tileBounds) )
        
        ### Need to also fix the coordinates of the polygon before I can use it as a mask
        overlapPoly = roiMask.intersection(tilePoly)
        
        ixfm = Affine.identity()
        s_xfm = [1,0,0,1,-1*tix,-1*tiy]  ## need to shim axis so I don't need to generate a giant numpy array that's 40kx40k
        poly_w_xfm = affine_transform(overlapPoly,s_xfm)

        roi_mask = rasterio.features.geometry_mask([poly_w_xfm],out_shape=(tile_info['height'],tile_info['width']),
                                                   transform=ixfm,invert=True)
        
        ## This version made the copied image gray..
        im1 = Image.fromarray(tile_info['tile'])
        im2 = Image.fromarray( backgroundtile)  ## Need to do this just once above
        im_mask = Image.fromarray(roi_mask)
        
        
        rgbimg = Image.new("RGBA", im2.size)
        rgbimg.paste(im2)
        im_c = Image.composite(im1, rgbimg, im_mask)
#         im_c = Image.composite(im1, im2, im_mask)
        sink.addTile( im_c,tix,tiy )
    else:
        ## Background area
        sink.addTile( backgroundtile, tix, tiy)
    num_tiles += 1

print('Number of tiles = {}'.format(num_tiles))

Number of tiles = 132


array([242.57216081, 242.65412763, 242.57955031, 255.        ])

In [35]:
sink.write("./maskedCaseDemoV4_added_white_at244_rgbamask.tiff")
#poly_w_xfm

In [ ]:
backgroundtile
# # for nparray, x, y in fancy_algorithm():
# #     # We could optionally add a mask to limit the output
# #     source.addTile(nparray, x, y)
# # source.write('/tmp/sample.tiff', lossy=False)
np.average(tile_info['tile'][:,:,:])


In [ ]:
#https://github.com/DigitalSlideArchive/HistomicsTK/blob/master/histomicstk/cli/utils.py

In [ ]:
poly, bound_box = intersectionList[2]

xfm = [1,0,0,1,-1*bound_box[0][0],-1*bound_box[0][1]]
poly_w_xfm = affine_transform(poly,xfm)


In [ ]:
# affinity_xfm = [1,0,0,1,0,0]
from rasterio import Affine


ixfm = Affine.identity()

roi_mask = rasterio.features.geometry_mask([poly_w_xfm],out_shape=(4000,4000),transform=ixfm)
plt.imshow(roi_mask)

In [ ]:
roi_mask

In [ ]:
list(poly_w_xfm.exterior.coords)

In [ ]:
sink = large_image.new()
# sink.addTile(numpydarray, x, y)
# sink.save()



In [ ]:

from skimage.draw import line, polygon, circle, ellipse
a_mask = np.ones((2000,2000),dtype="bool")
poly_c = np.array( intersectionList[0].exterior.coords)
rr, cc = polygon(poly_c[:,0], poly_c[:,1], (2000,2000))


In [ ]:
poly.exterior.bounds

In [ ]:
GeometryCollection(intersectionList)

In [ ]:
GeometryCollection(tileIntersect)
# baseSlidePoly.exterior.coords.xy ## Get the exterior hull coordinates for sanity check

In [ ]:
# baseSlidePoly.intersection(tilePoly)
GeometryCollection([tilePoly,baseSlidePoly])